<a href="https://colab.research.google.com/github/giseledoan/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-04 19:38:28--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-12-04 19:38:29 (6.40 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
#Load Amazon data into Spark DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["total_votes", "review_id", "star_rating", "helpful_votes", "vine", "verified_purchase"])
vine_df.show(5)

+-----------+--------------+-----------+-------------+----+-----------------+
|total_votes|     review_id|star_rating|helpful_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+----+-----------------+
|          0|R3I2DHQBR577SS|          5|            0|   N|                Y|
|          0|R1QNE9NQFJC2Y4|          5|            0|   N|                Y|
|          0|R3LIDG2Q4LJBAO|          5|            0|   N|                Y|
|          0|R3KSZHPAEVPEAL|          5|            0|   N|                Y|
|          0| RAI2OIG50KZ43|          5|            0|   N|                Y|
+-----------+--------------+-----------+-------------+----+-----------------+
only showing top 5 rows



In [15]:
# 1/ create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
votes20_df = vine_df.filter(vine_df.total_votes >= 20)
votes20_df.show(5)

+-----------+--------------+-----------+-------------+----+-----------------+
|total_votes|     review_id|star_rating|helpful_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+----+-----------------+
|         23|R2QRUE9REK8OUC|          5|           18|   N|                Y|
|         30|R2POXRW6PXHXZG|          5|           25|   N|                Y|
|         27| RZN43XRZ89IIJ|          3|           10|   N|                N|
|         24|R29Q748WSHZ3SN|          5|           23|   N|                Y|
|         20| R46UL5G5HEPRZ|          5|           18|   N|                N|
+-----------+--------------+-----------+-------------+----+-----------------+
only showing top 5 rows



In [16]:
# 2/ create a new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
votes_helpful_df = votes20_df.filter((votes_df.helpful_votes / votes_df.total_votes) >= 0.5)
votes_helpful_df.show(10)

+-----------+--------------+-----------+-------------+----+-----------------+
|total_votes|     review_id|star_rating|helpful_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+----+-----------------+
|         23|R2QRUE9REK8OUC|          5|           18|   N|                Y|
|         30|R2POXRW6PXHXZG|          5|           25|   N|                Y|
|         24|R29Q748WSHZ3SN|          5|           23|   N|                Y|
|         20| R46UL5G5HEPRZ|          5|           18|   N|                N|
|         76|R2UQHSNWU6WTZX|          5|           74|   N|                N|
|         71|R2HSRI3D6E2M9Y|          5|           69|   N|                Y|
|         29|R17ARFSDV555EW|          1|           29|   N|                Y|
|         21|R3GL5156FFEDQA|          5|           20|   N|                Y|
|         26|R29KQJC9PIYWQS|          5|           25|   N|                Y|
|         22| RYWUJT2A33VJH|          5|           20|   N|     

In [17]:
# 3/ create a new DataFrame that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
vine_yes_df = vine_df.filter(vine_df.vine == 'Y')
vine_yes_df.show(5)

+-----------+--------------+-----------+-------------+----+-----------------+
|total_votes|     review_id|star_rating|helpful_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+----+-----------------+
|          0| RD8F1K5G9W1VX|          3|            0|   Y|                N|
|          0|  R5TB74ZZY6HR|          4|            0|   Y|                N|
|          0|R1NALGILK9MRE5|          5|            0|   Y|                N|
|          0| ROTEP415SJ4DT|          5|            0|   Y|                N|
|          0|R2ZI2NSAZ0KBIB|          4|            0|   Y|                N|
+-----------+--------------+-----------+-------------+----+-----------------+
only showing top 5 rows



In [18]:
# 4/ retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'
vine_no_df = vine_df.filter(vine_df.vine == 'N')
vine_no_df.show(5)

+-----------+--------------+-----------+-------------+----+-----------------+
|total_votes|     review_id|star_rating|helpful_votes|vine|verified_purchase|
+-----------+--------------+-----------+-------------+----+-----------------+
|          0|R3I2DHQBR577SS|          5|            0|   N|                Y|
|          0|R1QNE9NQFJC2Y4|          5|            0|   N|                Y|
|          0|R3LIDG2Q4LJBAO|          5|            0|   N|                Y|
|          0|R3KSZHPAEVPEAL|          5|            0|   N|                Y|
|          0| RAI2OIG50KZ43|          5|            0|   N|                Y|
+-----------+--------------+-----------+-------------+----+-----------------+
only showing top 5 rows



In [19]:
# 5.1/ Total paid reviews
total_paid_reviews = vine_yes_df.count()
total_paid_reviews

33309

In [20]:
# 5.2/ Total paid reviews (5-star)
paid_five_star_reviews = vine_yes_df.filter(vine_yes_df.star_rating == 5).count()
paid_five_star_reviews

12429

In [21]:
# 5.3/ Percentage of paid reviews (5-star)
paid_five_star_percentage = (paid_five_star_reviews / total_paid_reviews) * 100
paid_five_star_percentage

37.314239394758175

In [22]:
# 5.4/ Total unpaid reviews
total_unpaid_reviews = vine_no_df.count()
total_unpaid_reviews

5082143

In [23]:
# 5.5/ Total upaid reviews (5-star)
unpaid_five_star_reviews = vine_no_df.filter(vine_no_df.star_rating == 5).count()
unpaid_five_star_reviews

3242324

In [24]:
# 5.6/ Percentage of unpaid reviews (5-star)
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100
unpaid_five_star_percent

63.79836222632854